# Hifi GAN

In [1]:
from nemo.collections.tts.models import HifiGanModel
model = HifiGanModel.from_pretrained(model_name="nvidia/tts_hifigan")

[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/megatron/core/tensor_parallel/layers.py:253: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/megatron/core/tensor_parallel/layers.py:264: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      @custom_bwd
    
[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/megatron/core/tensor_parallel/layers.py:324: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      @custom_fwd
    
[NeMo W 2024-09-22 17:15:50 nemo_logging:393] /home/lev

[NeMo I 2024-09-22 17:15:55 nemo_logging:381] PADDING: 0


[NeMo W 2024-09-22 17:15:55 nemo_logging:393] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2024-09-22 17:15:55 nemo_logging:381] PADDING: 0


[NeMo W 2024-09-22 17:15:55 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
      WeightNorm.apply(module, name, dim)
    
[NeMo W 2024-09-22 17:15:55 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer b

[NeMo I 2024-09-22 17:15:56 nemo_logging:381] Model HifiGanModel was successfully restored from /home/levi/.cache/huggingface/hub/models--nvidia--tts_hifigan/snapshots/3ba1fed954276287015654bf4c78060ffc9a4772/tts_hifigan.nemo.


# Generate Mel dataset

In [ ]:
import json
manifest = "./LJSpeech/valid/valid.json"
entries = []
with open(manifest, "r") as fjson:
    for line in fjson:
        entries.append(json.loads(line.strip()))

In [ ]:
# Load the Tacotron2Model
from nemo.collections.tts.models import Tacotron2Model
from nemo.collections.tts.models.base import SpectrogramGenerator
spec_generator = SpectrogramGenerator.from_pretrained("tts_en_tacotron2")

In [48]:
def __generate_mels(entry, spec_model, device):
    audio = __load_wav(entry["audio_filepath"]).to(device)  # Load audio and move to the device
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    with torch.no_grad():
        parsed = spec_model.parse(entry['text'])
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
        spect = spec_model.generate_spectrogram(tokens=parsed)

        audio_path = entry['audio_filepath'][22:][:-4]
        mel_path = f'./meldataset/valid/{audio_path}.npy'
        np.save(mel_path, spect[0].to('cpu').detach().numpy())

for entry in entries:
    __generate_mels(entry, spec_model, device)

## Fast Pitch


In [51]:
from nemo.collections.tts.parts.utils.tts_dataset_utils import (
    BetaBinomialInterpolator,
    beta_binomial_prior_distribution,
)
def __load_wav(audio_file):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
    return samples.transpose()


def __generate_mels(entry, spec_model, device, train, use_beta_binomial_interpolator):
    # Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
    audio = __load_wav(entry["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)

    with torch.no_grad():
        text = spec_model.parse(entry['text'])
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        if use_beta_binomial_interpolator:
            beta_binomial_interpolator = BetaBinomialInterpolator()
            attn_prior = (
                torch.from_numpy(beta_binomial_interpolator(spect_len.item(), text_len.item()))
                .unsqueeze(0)
                .to(text.device)
            )
        else:
            attn_prior = (
                torch.from_numpy(beta_binomial_prior_distribution(text_len.item(), spect_len.item()))
                .unsqueeze(0)
                .to(text.device)
            )

        spectrogram = spec_model.forward(
            text=text, input_lens=text_len, spec=spect, mel_lens=spect_len, attn_prior=attn_prior
        )[0]
        if train:
            mel_path = f'./meldataset-fastpitch/train/{audio_path}.npy'
        else:
            mel_path = f'./meldataset-fastpitch/valid/{audio_path}.npy'
        np.save(mel_path, spectrogram[0].to('cpu').detach().numpy())
        entry["mel_filepath"] = str(mel_path)
    return  entry


In [ ]:
from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.from_pretrained("tts_en_fastpitch")

In [54]:
text = spec_model.parse(entry['text'])
text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
spect, spect_len = spec_generator.preprocessor(input_signal=audio, length=audio_len)

[NeMo W 2024-09-23 16:35:08 nemo_logging:393] /home/levi/Speech-Course-Lab/env/lib/python3.11/site-packages/nemo/collections/asr/parts/preprocessing/features.py:417: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      with torch.cuda.amp.autocast(enabled=False):
    


In [55]:
use_beta_binomial_interpolator = True
if use_beta_binomial_interpolator:
    beta_binomial_interpolator = BetaBinomialInterpolator()
    attn_prior = (
        torch.from_numpy(beta_binomial_interpolator(spect_len.item(), text_len.item()))
        .unsqueeze(0)
        .to(text.device)
    )
else:
    attn_prior = (
        torch.from_numpy(beta_binomial_prior_distribution(text_len.item(), spect_len.item()))
        .unsqueeze(0)
        .to(text.device)
    )

In [59]:
spectrogram = spec_generator.forward(
    text=text, input_lens=text_len, spec=spect, mel_lens=spect_len, attn_prior=attn_prior
)[0]
spectrogram

tensor([[[-7.6340, -6.8896, -6.3471,  ..., -7.7915, -7.9398, -7.6211],
         [-7.2663, -6.5574, -5.7835,  ..., -7.0151, -7.1644, -7.1354],
         [-7.0367, -6.1788, -5.2900,  ..., -6.5786, -6.6716, -6.8804],
         ...,
         [-7.4785, -6.4077, -6.0738,  ..., -6.7966, -7.3465, -7.5854],
         [-7.5187, -6.2881, -5.9374,  ..., -6.9806, -7.4701, -7.7482],
         [-7.6504, -6.1538, -5.7567,  ..., -7.2051, -7.7050, -8.0217]]],
       grad_fn=<TransposeBackward0>)

In [ ]:
import json
train_entries = []
with open("./LJSpeech/train/train.json", "r") as fjson:
    for line in fjson:
        entries.append(json.loads(line.strip()))
__generate_mels(entries, spec_model, 'cpu', True, True)

In [ ]:
!(python examples/tts/hifigan_finetune.py \
--config-name=hifigan.yaml \
model.train_ds.dataloader_params.batch_size=32 \
model.max_steps=1000 \
model.optim.lr=0.00001 \
~model.optim.sched \
train_dataset={hifigan_train_ds} \
validation_datasets={hifigan_val_ds} \
exp_manager.exp_dir={os.environ["RESULTS_DIR"]} \
+init_from_pretrained_model=tts_hifigan \
trainer.check_val_every_n_epoch=10 \
model/train_ds=train_ds_finetune \
model/validation_ds=val_ds_finetune)

# Training Hifi-GAN

In [ ]:
import pytorch_lightning as pl

from nemo.collections.tts.models import HifiGanModel
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager


@hydra_runner(config_path="conf/hifigan", config_name="hifigan")
def main(cfg):
    trainer = pl.Trainer(**cfg.trainer)
    exp_manager(trainer, cfg.get("exp_manager", None))
    model = HifiGanModel(cfg=cfg.model, trainer=trainer)
    trainer.fit(model)